In [1]:
from qiskit import QuantumCircuit
from typing import List
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from collections import Counter

from bloqade import qasm2
from kirin.dialects import ilist
from pyqrack import QrackSimulator
from bloqade.pyqrack import PyQrack, reg
from bloqade.noise import native


from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

In [2]:
def to_bitstrings(results):
    return Counter(map(lambda result:"".join(map(str, result)), results))

# Let's start with repetition code

Here we use the bloqade and kirin annotation to implement the repetition code

In [3]:
@qasm2.extended
def repetition_code():
    
    qreg = qasm2.qreg(5)
    creg = qasm2.creg(5)
    '''
    Srror propagation and syndrome extraction
    '''
    qasm2.cx(qreg[0], qreg[3])
    qasm2.cx(qreg[1], qreg[3])
    qasm2.cx(qreg[1], qreg[4])
    qasm2.cx(qreg[2], qreg[4])
    
    qasm2.measure(qreg[0], creg[0])
    qasm2.measure(qreg[1], creg[1])
    qasm2.measure(qreg[2], creg[2])
    qasm2.measure(qreg[3], creg[3])
    qasm2.measure(qreg[4], creg[4])
    '''
    Decoding and error correction
    '''
    if creg[3] == 0 and creg[4] == 1:
        qasm2.x(qreg[2])    
    
    if creg[3] == 1 and creg[4] == 1:
        qasm2.x(qreg[1])
        
    if creg[3] == 1 and creg[4] == 0:
        qasm2.x(qreg[0])    
        
    return creg

### Now, since there are no quantum noise, if we take 100 shots, we will always get 0s

In [4]:
device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
results = device.multi_run(repetition_code, _shots=100)

counts = to_bitstrings(results)

for key, value in counts.items():
    print(key, value)

00000 100


### We manualy inject some pauli x noise to check if the repeition code work or not 

In [5]:


def repetition_inject_noise_simulation():

    @qasm2.extended
    def repetition_noise_inject(qreg: qasm2.QReg,creg:qasm2.CReg,xnoiseindex:tuple[int, ...]):
        
        for i in range(len(xnoiseindex)):
            qasm2.x(qreg[xnoiseindex[i]])    
        
        qreg = qasm2.qreg(5)
        creg = qasm2.creg(5)
        '''
        Srror propagation and syndrome extraction
        '''
        qasm2.cx(qreg[0], qreg[3])
        qasm2.cx(qreg[1], qreg[3])
        qasm2.cx(qreg[1], qreg[4])
        qasm2.cx(qreg[2], qreg[4])
        
        qasm2.measure(qreg[0], creg[0])
        qasm2.measure(qreg[1], creg[1])
        qasm2.measure(qreg[2], creg[2])
        qasm2.measure(qreg[3], creg[3])
        qasm2.measure(qreg[4], creg[4])
        '''
        Decoding and error correction
        '''
        if creg[3] == 0 and creg[4] == 1:
            qasm2.x(qreg[2])    
        
        if creg[3] == 1 and creg[4] == 1:
            qasm2.x(qreg[1])
            
        if creg[3] == 1 and creg[4] == 0:
            qasm2.x(qreg[0])    
            
        return creg     
    
    @qasm2.extended
    def run_insert_noise_program():
        qreg=qasm2.qreg(5)
        creg=qasm2.creg(5)
        repetition_noise_inject(qreg,creg,(0,1,2))
        return creg

    return run_insert_noise_program
            

In [6]:
device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
results = device.multi_run(repetition_inject_noise_simulation(), _shots=100)

counts = to_bitstrings(results)

for key, value in counts.items():
    print(key, value)

00000 100


# Our implementation of Surface code

We implement the surface code class first. The goal of this class is to calculate the coordination of all qubits in the code. So it would latter be more convenient for use to develop it in bloqade

In [7]:
class stabalizer:
    
    '''
    The stabilzier is a list of typle.
    For example:
          stab=[('X',1),('X',2),('Z',5),('Z',6)] 
          means the stabilizer is X1-X2-Z5-Z6
    '''
    def __init__(self,stab:List[tuple])->None:
        self.__stab=stab


    def __str__(self) -> str:
        strresult=""
        index=0
        for (stype,qubit) in self.__stab:
            strresult=strresult+stype+str(qubit)
            if index<len(self.__stab)-1:
                strresult=strresult+"-"
            index=index+1
        return strresult
    
    
    def get_stab(self)->List[tuple]:
        return self.__stab


class surfaceCode:
    
    
    '''
    Initialize a d*d surface code
    '''
    def __init__(self,distance:int) -> None:
        self._distance=distance
        '''
        A standard surface code has 2*distance**2-1 qubits and distance**2-1 stabilizers
        Following is the dataqubit of a 4*4 surface code                                                                                    
                                    Q1----Q2----Q3----Q4
                                    |    |     |     |
                                    |    |     |     | 
                                    Q5----Q6----Q7----Q8
                                    |    |     |     |
                                    |    |     |     | 
                                    Q9---Q10---Q11---Q12
                                    |    |     |     | 
                                    |    |     |     |
                                    Q13--Q14---Q15---Q16   
        We use the standard coordinate system to represent the qubits.
        Q1: (0,0), Q2: (0,1), Q3: (0,2), Q4: (0,3)                      
        Q5: (1,0), Q6: (1,1), Q7: (1,2), Q8: (1,3)
        Q9: (2,0), Q10:(2,1), Q11:(2,2), Q12:(2,3)
        Q13:(3,0), Q14:(3,1), Q15:(3,2), Q16:(3,3)                                     
        '''
        self._circuit=QuantumCircuit(2*distance**2-1,distance**2-1)
        self._ndataqubits=distance**2
        self._nstab=distance**2-1
        self._nXstab=0
        self._nZstab=0
        self._stab=[]
        self._error={}
        self.calc_stab()
        self._Hmatrix=np.zeros((self._nstab, 2*self._ndataqubits),dtype=int)
        self.calculate_H_matrix()
        
    def get_xy(self,qubit:int)->tuple:
        return qubit//self.__distance,qubit%self.__distance
        
        
    def draw_surface(self)->None:
        resultstr=""
        for line in range(self._distance):
            linestr=""
            for column in range(self._distance):
                index=line*self._distance+column+1
                if(column<self._distance-1):
                    if(len(str(index))==1):
                        linestr=linestr+"Q"+str(index)+"----"
                    if(len(str(index))==2):
                        linestr=linestr+"Q"+str(index)+"---"
                else:
                    linestr=linestr+"Q"+str(index)

            if line<self._distance-1:
                linestr=linestr+"\n"
                for column in range(self._distance-1):
                    linestr=linestr+"|     "
                linestr=linestr+"|\n"
                for column in range(self._distance-1):
                    linestr=linestr+"|     "
                linestr=linestr+"|\n"       
            resultstr=resultstr+linestr
        print(resultstr)     
        
    '''
    Calculate the satbilizer
    Each stabilizer is stored in a list.
    For example, S1=[1,2,5,6]
    '''    
    def calc_stab(self):
        '''
        Add X stabilizers in side the boundary
                        Qa------Qa+1
                        |       |
                        |       |
                        Qa+d----Qa+d+1
        line 0: Q1-Q2-Q3-...-Qd
        line 1: Qd+1-Q6-Q7-Q8-...-Q2d
        ...
        line k: Qkd+1-Qkd+2-...-Q(k+1)d
                        
        '''
        d=self._distance
        currentLine=0
        lefttopIndex=1
        while currentLine<d  and lefttopIndex+d+1<=d**2:
            self._stab.append(stabalizer([('X',lefttopIndex),('X',lefttopIndex+1),('X',lefttopIndex+d),('X',lefttopIndex+d+1)]))
            self._nXstab+=1
            if lefttopIndex+2 < (currentLine+1)*d:
                lefttopIndex=lefttopIndex+2
            else:
                currentLine=currentLine+1
                lefttopIndex=lefttopIndex+3
    
        '''
        Add X stabilizers that attached to the top and bottom boundary(Tough boundary)
        '''        
        leftindex=2
        while leftindex+1<=d:
            self._stab.append(stabalizer([('X',leftindex),('X',leftindex+1)]))
            self._nXstab+=1
            leftindex+=2
        leftindex=d*(d-1)+2
        while leftindex+1<=d**2:
            self._stab.append(stabalizer([('X',leftindex),('X',leftindex+1)]))
            self._nXstab+=1
            leftindex+=2
                            
        '''
        Add Z stabilizers inside the boundary
        '''
        currentLine=0
        lefttopIndex=2
        while currentLine<d  and lefttopIndex+d+1<=d**2:
            self._stab.append(stabalizer([('Z',lefttopIndex),('Z',lefttopIndex+1),('Z',lefttopIndex+d),('Z',lefttopIndex+d+1)]))
            self._nZstab+=1
            if lefttopIndex+2 < (currentLine+1)*d:
                lefttopIndex=lefttopIndex+2
            else:
                currentLine=currentLine+1
                lefttopIndex=lefttopIndex+3       

        '''
        Add Z stabilizers that attached to the left and right boundary(Soft boundary)
        '''          
        topindex=1     
        while topindex+d<=(d-1)*d+1:
            self._stab.append(stabalizer([('Z',topindex),('Z',topindex+d)]))
            self._nZstab+=1
            topindex=topindex+2*d
        topindex=d
        while topindex+d<=d**2:
            self._stab.append(stabalizer([('Z',topindex),('Z',topindex+d)]))
            self._nZstab+=1
            topindex=topindex+2*d


    def get_stab_by_index(self,index:int)->stabalizer:
        return self._stab[index]
    
    
    def print_stab(self):
        for s in self._stab:
            print(s)
    

    
    def compile_syndrome_circuit(self):
        synindex=0
        for stab in self._stab:
            tmpstab=stab.get_stab()
            if tmpstab[0][0]=='X':
                self._circuit.h(self._ndataqubits+synindex)
                for (typestr,qubit) in tmpstab:
                        self._circuit.cx(self._ndataqubits+synindex,qubit)
                
                self._circuit.h(self._ndataqubits+synindex)
            else:
                
                for (typestr,qubit) in tmpstab:
                        self._circuit.cx(qubit,self._ndataqubits+synindex)                     
            '''
            Measure the syndrome qubit
            ''' 
            self._circuit.measure(self._ndataqubits+synindex,synindex)
            synindex+=1 
        
        
    '''
    Inject error to the data qubit of the surface code
    Error is a dictionary, the key is the qubit index, the value is the error type
    For example, error={1:'X',2:'Z',3:'Y'} means qubit 1 has X error, qubit 2 has Z error, qubit 3 has Y error
    '''
    def inject_error(self,error:dict):
        self._error=error
        for qubit in error:
            if error[qubit]=='X':
                self._circuit.x(qubit)
            if error[qubit]=='Z':
                self._circuit.z(qubit)
            if error[qubit]=='Y':
                self._circuit.y(qubit)
    
             
        
        
    def get_circuit(self)->QuantumCircuit:
        return self._circuit
        
        
        
    '''
    Surface code is a CSS code, which means it has X and Z stabilizers.
    We can calculate the check matrix H
    For example, a code with stabilizers: S1=Z1Z2, S2=Z2Z3, S3=X1X2, S4=X2X3 has the following check matrix:
                                Z1  Z2  Z3  X1  X2  X3                                       
                            S1  1   1   0   0   0   0 
                        H=  S2  0   1   1   0   0   0   
                            S3  0   0   0   1   1   0 
                            S4  0   0   0   0   1   1 
                        
    '''                     
    
    def calculate_H_matrix(self):
        stabindex=0
        for stab in self._stab:
            tmpstab=stab.get_stab()
            for (typestr,qubit) in tmpstab:
                if typestr=='Z':
                    self._Hmatrix[stabindex][qubit-1]=1
                else:
                    self._Hmatrix[stabindex][self._ndataqubits+qubit-1]=1
            stabindex+=1
    
    
        
    def get_check_matrix(self)->np.array:
        return self._Hmatrix
    

    def run_simulation(self,shots:int)->dict:
        backend = AerSimulator()
        job = backend.run(self._circuit, shots=shots)
        output = job.result().get_counts() 
        return output
    

Initialize a surface code with code distance 3, print to see the structure and stabilizers

In [8]:
suf=surfaceCode(3)
suf.draw_surface()
suf.print_stab()

Q1----Q2----Q3
|     |     |
|     |     |
Q4----Q5----Q6
|     |     |
|     |     |
Q7----Q8----Q9
X1-X2-X4-X5
X4-X5-X7-X8
X2-X3
X8-X9
Z2-Z3-Z5-Z6
Z5-Z6-Z8-Z9
Z1-Z4
Z3-Z6


## Now we know the stabilzer of the circuit, we can implement it in bloquade

Idea of implementation: First, because surface code is a CSS code with only Z or X stabilizer, we create two helper function which compiler and generate the stabilizer measurement for any stabilizer.

In [9]:
@qasm2.extended
def add_X_syndrome_circuit(qreg: qasm2.QReg,creg:qasm2.CReg,ndataqubits:int,stabindex:int,index_tuple:tuple[int, ...]):
    qasm2.h(qreg[ndataqubits+stabindex])
    for i in range(len(index_tuple)):
        qasm2.cx(qreg[ndataqubits+stabindex],qreg[index_tuple[i]])
    qasm2.h(qreg[ndataqubits+stabindex])
    qasm2.measure(qreg[ndataqubits+stabindex],creg[stabindex])



@qasm2.extended
def add_Z_syndrome_circuit(qreg: qasm2.QReg,creg:qasm2.CReg,ndataqubits:int,stabindex:int,index_tuple:tuple[int, ...]):
    for i in range(len(index_tuple)):
        qasm2.cx(qreg[index_tuple[i]],qreg[ndataqubits+stabindex])
    qasm2.measure(qreg[ndataqubits+stabindex],creg[stabindex])



'''
@qasm2.extended
def add_X_syndrome_circuit_recursive(result:int,round:int,qreg: qasm2.QReg,creg:qasm2.CReg,ndataqubits:int,stabindex:int,index_tuple:tuple[int, ...]):
    if round==3:
        return
    
    qasm2.h(qreg[ndataqubits+stabindex])
    for i in range(len(index_tuple)):
        qasm2.cx(qreg[ndataqubits+stabindex],qreg[index_tuple[i]])
    qasm2.h(qreg[ndataqubits+stabindex])
    qasm2.measure(qreg[ndataqubits+stabindex],creg[stabindex])


    if creg[stabindex]==result:
        return add_X_syndrome_circuit_recursive(result,round+1,qreg,creg,ndataqubits,stabindex,index_tuple)
       
    qasm2.reset(qreg[ndataqubits+stabindex]) 
    return add_X_syndrome_circuit_recursive(1-result,1,qreg,creg,ndataqubits,stabindex,index_tuple)


@qasm2.extended
def add_Z_syndrome_circuit_recursive(result:int,round:int,qreg: qasm2.QReg,creg:qasm2.CReg,ndataqubits:int,stabindex:int,index_tuple:tuple[int, ...]):
    if round==3:
        return
    
    for i in range(len(index_tuple)):
        qasm2.cx(qreg[index_tuple[i]],qreg[ndataqubits+stabindex])
    qasm2.measure(qreg[ndataqubits+stabindex],creg[stabindex])
    
    if creg[stabindex]==result:
        return add_Z_syndrome_circuit_recursive(result,round+1,qreg,creg,ndataqubits,stabindex,index_tuple)
   
    qasm2.reset(qreg[ndataqubits+stabindex]) 
    return add_Z_syndrome_circuit_recursive(1-result,1,qreg,creg,ndataqubits,stabindex,index_tuple)
'''




'''
@qasm2.extended
def surface_code_d2_circuit_recursive():
    qreg = qasm2.qreg(2*3**2-1)
    creg = qasm2.creg(3**2-1)
    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,0,(1,2,4,5))
    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,1,(4,5,7,8))
    
    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,2,(2,3))    
    add_X_syndrome_circuit_recursive(qreg,creg,9,3,(8,9))    
    
    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,4,(2,3,5,6))
    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,5,(5,6,8,9))
    
    
    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,6,(1,4))    
    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,7,(3,6))       
'''

'\n@qasm2.extended\ndef surface_code_d2_circuit_recursive():\n    qreg = qasm2.qreg(2*3**2-1)\n    creg = qasm2.creg(3**2-1)\n    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,0,(1,2,4,5))\n    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,1,(4,5,7,8))\n\n    add_X_syndrome_circuit_recursive(0,0,qreg,creg,9,2,(2,3))    \n    add_X_syndrome_circuit_recursive(qreg,creg,9,3,(8,9))    \n\n    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,4,(2,3,5,6))\n    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,5,(5,6,8,9))\n\n\n    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,6,(1,4))    \n    add_Z_syndrome_circuit_recursive(0,0,qreg,creg,9,7,(3,6))       \n'

The distance 3 surface code memory with one round of symdrome is defined as follows, here we neglect the decoding algorithm. 

In [10]:

@qasm2.extended
def surface_code_d3_circuit():
    qreg = qasm2.qreg(2*3**2-1)
    creg = qasm2.creg(3**2-1)
    add_X_syndrome_circuit(qreg,creg,9,0,(0,1,3,4))
    add_X_syndrome_circuit(qreg,creg,9,1,(3,4,6,7))
    
    add_X_syndrome_circuit(qreg,creg,9,2,(1,2))    
    add_X_syndrome_circuit(qreg,creg,9,3,(7,8))    
    
    add_Z_syndrome_circuit(qreg,creg,9,4,(1,2,4,5))
    add_Z_syndrome_circuit(qreg,creg,9,5,(4,5,7,8))
    
    
    add_Z_syndrome_circuit(qreg,creg,9,6,(0,3))    
    add_Z_syndrome_circuit(qreg,creg,9,7,(2,5))          


Now, we can print the qasm2 file of the surface code memory, which is enough as the input for the next level of compilation and optimization. 

In [11]:
target = QASM2()
ast = target.emit(surface_code_d3_circuit)
pprint(ast)

OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[17];
creg creg[8];
h qreg[9];
CX qreg[9], qreg[0];
CX qreg[9], qreg[1];
CX qreg[9], qreg[3];
CX qreg[9], qreg[4];
h qreg[9];
measure qreg[9] -> creg[0];
h qreg[10];
CX qreg[10], qreg[3];
CX qreg[10], qreg[4];
CX qreg[10], qreg[6];
CX qreg[10], qreg[7];
h qreg[10];
measure qreg[10] -> creg[1];
h qreg[11];
CX qreg[11], qreg[1];
CX qreg[11], qreg[2];
h qreg[11];
measure qreg[11] -> creg[2];
h qreg[12];
CX qreg[12], qreg[7];
CX qreg[12], qreg[8];
h qreg[12];
measure qreg[12] -> creg[3];
CX qreg[1], qreg[13];
CX qreg[2], qreg[13];
CX qreg[4], qreg[13];
CX qreg[5], qreg[13];
measure qreg[13] -> creg[4];
CX qreg[4], qreg[14];
CX qreg[5], qreg[14];
CX qreg[7], qreg[14];
CX qreg[8], qreg[14];
measure qreg[14] -> creg[5];
CX qreg[0], qreg[15];
CX qreg[3], qreg[15];
measure qreg[15] -> creg[6];
CX qreg[2], qreg[16];
CX qreg[5], qreg[16];
measure qreg[16] -> creg[7];


# Simple logical algorithm

We also implement a surface logical circuit with one logical X, one logical CNOT, and a logical measurement. 

In [12]:

@qasm2.extended
def surface_code_d3_transversal_cnot():
    qreg = qasm2.qreg(2*2*3**2-2)
    creg = qasm2.creg(2*3**2-2+3)
    
    
    for i in range(2*3**2-2):
        qasm2.reset(qreg[i])  
    
    '''
    Logical X gate
    '''
    qasm2.x(qreg[9])
    qasm2.x(qreg[10])
    qasm2.x(qreg[11])        
   

    '''
    Logical transversal CNOT gate
    '''
    for i in range(9):
        qasm2.cx(qreg[i],qreg[i+9])


    '''
    Syndrome measurement on first logical qubit
    '''
    add_X_syndrome_circuit(qreg,creg,18,0,(0,1,3,4))
    add_X_syndrome_circuit(qreg,creg,18,1,(3,4,6,7))
    
    add_X_syndrome_circuit(qreg,creg,18,2,(1,2))    
    add_X_syndrome_circuit(qreg,creg,18,3,(7,8))    
    
    add_Z_syndrome_circuit(qreg,creg,18,4,(1,2,4,5))
    add_Z_syndrome_circuit(qreg,creg,18,5,(4,5,7,8))
    
    
    add_Z_syndrome_circuit(qreg,creg,18,6,(0,3))    
    add_Z_syndrome_circuit(qreg,creg,18,7,(2,5))             

    '''
    Syndrome measurement on second logical qubit
    '''
    add_X_syndrome_circuit(qreg,creg,18,8,(9,10,12,13))
    add_X_syndrome_circuit(qreg,creg,18,9,(12,13,15,16))
    
    add_X_syndrome_circuit(qreg,creg,18,10,(10,11))    
    add_X_syndrome_circuit(qreg,creg,18,11,(16,17))    
    
    add_Z_syndrome_circuit(qreg,creg,18,12,(10,11,13,14))
    add_Z_syndrome_circuit(qreg,creg,18,13,(13,14,16,17))
    
    
    add_Z_syndrome_circuit(qreg,creg,18,14,(9,12))    
    add_Z_syndrome_circuit(qreg,creg,18,15,(11,14))  

    '''
    Measuring the logical qubit
    '''
    qasm2.measure(qreg[9],creg[16])
    qasm2.measure(qreg[10],creg[17])
    qasm2.measure(qreg[11],creg[18])      
    return creg
    

In [13]:
target = QASM2()
ast = target.emit(surface_code_d3_transversal_cnot)
pprint(ast)

OPENQASM 2.0;
include "qelib1.inc";
qreg qreg[34];
creg creg[19];
reset qreg[0];
reset qreg[1];
reset qreg[2];
reset qreg[3];
reset qreg[4];
reset qreg[5];
reset qreg[6];
reset qreg[7];
reset qreg[8];
reset qreg[9];
reset qreg[10];
reset qreg[11];
reset qreg[12];
reset qreg[13];
reset qreg[14];
reset qreg[15];
x qreg[9];
x qreg[10];
x qreg[11];
CX qreg[0], qreg[9];
CX qreg[1], qreg[10];
CX qreg[2], qreg[11];
CX qreg[3], qreg[12];
CX qreg[4], qreg[13];
CX qreg[5], qreg[14];
CX qreg[6], qreg[15];
CX qreg[7], qreg[16];
CX qreg[8], qreg[17];
h qreg[18];
CX qreg[18], qreg[0];
CX qreg[18], qreg[1];
CX qreg[18], qreg[3];
CX qreg[18], qreg[4];
h qreg[18];
measure qreg[18] -> creg[0];
h qreg[19];
CX qreg[19], qreg[3];
CX qreg[19], qreg[4];
CX qreg[19], qreg[6];
CX qreg[19], qreg[7];
h qreg[19];
measure qreg[19] -> creg[1];
h qreg[20];
CX qreg[20], qreg[1];
CX qreg[20], qreg[2];
h qreg[20];
measure qreg[20] -> creg[2];
h qreg[21];
CX qreg[21], qreg[7];
CX qreg[21], qreg[8];
h qreg[21];
measure q

# Testing

In order to test the correctness of the code, we manually insert some X,Z pauli noise. 

In [ ]:
@qasm2.extended
def surface_code_d3_circuit_noise_inject(qreg: qasm2.QReg,creg:qasm2.CReg,xnoiseindex:tuple[int, ...],znoiseindex:tuple[int, ...]):
    '''
    First, apply syndrome measurement as the reference
    '''
    add_X_syndrome_circuit(qreg,creg,9,0,(1,2,4,5))
    add_X_syndrome_circuit(qreg,creg,9,1,(4,5,7,8))
    

    
    add_X_syndrome_circuit(qreg,creg,9,2,(2,3))    
    add_X_syndrome_circuit(qreg,creg,9,3,(8,9))    
    
    

    add_Z_syndrome_circuit(qreg,creg,9,4,(2,3,5,6))
    add_Z_syndrome_circuit(qreg,creg,9,5,(5,6,8,9))
    
    
    add_Z_syndrome_circuit(qreg,creg,9,6,(1,4))    
    add_Z_syndrome_circuit(qreg,creg,9,7,(3,6))      

    
    
    for i in range(len(xnoiseindex)):
        qasm2.x(qreg[xnoiseindex[i]])
    
    for i in range(len(znoiseindex)):
        qasm2.z(qreg[znoiseindex[i]])
       
        
    '''
    The second syndrome measurement is used ,to check the error, it should be consistent with the first one.
    '''
    
    
    add_X_syndrome_circuit(qreg,creg,9,8,(1,2,4,5))
    add_X_syndrome_circuit(qreg,creg,9,9,(4,5,7,8))
    
    add_X_syndrome_circuit(qreg,creg,9,10,(2,3))    
    add_X_syndrome_circuit(qreg,creg,9,11,(8,9))    
    
    add_Z_syndrome_circuit(qreg,creg,9,12,(2,3,5,6))
    add_Z_syndrome_circuit(qreg,creg,9,13,(5,6,8,9))
    
    
    add_Z_syndrome_circuit(qreg,creg,9,14,(1,4))    
    add_Z_syndrome_circuit(qreg,creg,9,15,(3,6))   
          

        

In [ ]:
@qasm2.extended
def inject_noise_simulation():
    qreg = qasm2.qreg(2*3**2-1+8)
    creg = qasm2.creg(3**2-1+8) 
    
    for i in range(2*3**2-1):
        qasm2.reset(qreg[i])

    surface_code_d3_circuit_noise_inject(qreg,creg,(1,),(2,) )# Add x error to qubit 1 but not z error
    
    return creg

The first set of the measurement and the second set of the measurement should match, if they don't, we know that something happen, 
we can print these detector events:

In [18]:
device = PyQrack(dynamic_qubits=True, pyqrack_options={"isBinaryDecisionTree": False})
results = device.multi_run(inject_noise_simulation, _shots=100)

counts = to_bitstrings(results)


for key, value in counts.items():
    firsthalf=key[0:len(key)//2]
    secondhalf=key[len(key)//2:len(key)]
    detectorstr=""
    for i in range(len(firsthalf)):
        if firsthalf[i]!=secondhalf[i]:
            detectorstr+="1"
        else:
            detectorstr+="-"
    print(detectorstr, value)



-----1-- 1
-1--1111 1
11------ 1
1-1--1-- 1
-11----1 1
--1-1-11 1
-11--1-- 1
-11----1 1
-11----- 1
1-1----1 1
1-1-1-1- 1
-1---1-1 1
-1--1-11 1
-1--1111 1
111-1-11 1
111--1-1 1
1------1 1
11--1111 1
111----- 1
1-1-1-11 1
11------ 1
-1---1-- 1
1-1-1111 1
1-1-1-1- 1
-1-----1 1
11--1-11 1
1----1-- 1
111-1-1- 1
--1--1-- 1
111-1-1- 1
111-1-1- 1
--1----- 1
--1----1 1
111-1-1- 1
1-1-1-11 1
-11--1-1 1
111--1-1 1
11--1-1- 1
--1--1-- 1
11-----1 1
-11-1-11 1
-------1 1
1-1-1-11 1
-1-----1 1
-11--1-1 1
11---1-- 1
-11--1-- 1
1------1 1
--1----1 1
11--1-1- 1
111-1111 1
-11-1111 1
--1--1-1 1
--1-1111 1
--1-1111 1
-----1-- 1
--1-1-1- 1
11------ 1
11--1-11 1
--1-1111 1
1-1-1-11 1
1---1-11 1
1---1-1- 1
-11----- 1
11-----1 1
-----1-1 1
--1-1111 1
--1-1111 1
-1-----1 1
-1--111- 1
111-1111 1
-11-1-11 1
-11----1 1
1-1--1-- 1
-1------ 1
1-1-1-1- 1
1---1111 1
-1--111- 1
1----1-1 1
1-1----1 1
-11-111- 1
-1--1111 1
----1-1- 1
111-1-11 1
11--111- 1
-----1-- 1
111-111- 1
111-1111 1
1-1----1 1
--1-1111 1
-11----1 1